Họ và tên: Nguyễn Thị Huyền Trang

MSSV: 21520488


# **LAB 03 - KHUYẾN NGHỊ DỰA TRÊN NỘI DUNG**

In [ ]:
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

pd.options.display.max_colwidth = 200
sns.set()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#Load dữ liệu movies_metadata.csv
movies = pd.read_csv('/content/drive/MyDrive/Hệ khuyến nghị - DS300.P11/Lab03/dataset/movies_metadata.csv')

<ipython-input-3-2ba472ef636c>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('/content/drive/MyDrive/Hệ khuyến nghị - DS300.P11/Lab03/dataset/movies_metadata.csv')


In [ ]:
# Xuất ra overview của 5 bộ phim đầu tiên
movies[['overview']].head(5)

,overview
0,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circum..."
1,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- in..."
2,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming t..."
3,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, ..."
4,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home..."


In [ ]:
# Viết hàm tiền xử lý dữ liệu trên cột overview
def processing_data(data):
  # Chuyển về từ thường
  data = data.lower()
  # Xóa dấu câu, ký tự đặc biệt
  data = re.sub('\W+',' ', data)
  # Xóa khoảng trắng đầu và cuối câu
  data = data.strip()
  # Xóa stopword
  data = ' '.join([word for word in data.split() if word not in stopwords.words("english")])
  # Tách từ
  # word_tokens = word_tokenize(data)
  # ...
  return data

In [ ]:
# Xử lý dữ liệu trên cột overview
processing_data(movies['overview'])

  0%|          | 0/45466 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
100%|██████████| 45466/45466 [06:48<00:00, 111.34it/s]


0        led woody andy toys live happily room andy birthday brings buzz lightyear onto scene afraid losing place andy heart woody plots buzz circumstances separate buzz woody owner duo eventually learns p...
1        siblings judy peter discover enchanted board game opens door magical world unwittingly invite alan adult trapped inside game 26 years living room alan hope freedom finish game proves risky three f...
2        family wedding reignites ancient feud next door neighbors fishing buddies john max meanwhile sultry italian divorcée opens restaurant local bait shop alarming locals worry scare fish away less int...
3                   cheated mistreated stepped women holding breath waiting elusive good man break string less stellar lovers friends confidants vannah bernie glo robin talk determined find better way breathe
4                                                 george banks recovered daughter wedding receives news pregnant george wife nina expecting planning selling home pl

In [ ]:
# Sử dụng TfidfVectorizer để chuyển cột overview của các bộ phim về tf-idf và lưu vào biến overview_matrix
vectorizer = TfidfVectorizer(stop_words='english')
overview_matrix = vectorizer.fit_transform(movies['overview'])

In [ ]:
# In ra shape của overview_matrix nếu = (45466, 75827) là đúng.
print(overview_matrix.shape)

(45466, 75801)


In [ ]:
# Tính toán cosine giữa các bộ phim với nhau bằng linear_kernel
cosine_sim = linear_kernel(overview_matrix, overview_matrix)
cosine_sim

array([[0.58098714, 0.        , 0.        , ..., 0.        , 0.01757945,
        0.        ],
       [0.        , 0.61125382, 0.02200543, ..., 0.03636249, 0.        ,
        0.        ],
       [0.        , 0.02200543, 0.46607905, ..., 0.        , 0.00649986,
        0.00636462],
       ...,
       [0.        , 0.03636249, 0.        , ..., 0.6609658 , 0.        ,
        0.        ],
       [0.01757945, 0.        , 0.00649986, ..., 0.        , 0.39788609,
        0.00550112],
       [0.        , 0.        , 0.00636462, ..., 0.        , 0.00550112,
        0.52920617]])

In [ ]:
# Đánh index cho các bộ phim bằng pd.Series() và lưu trong biến mapping
mapping = pd.Series(movies.index, index=movies['title']).drop_duplicates()
mapping

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Subdue                         45461
Century of Birthing            45462
Betrayal                       45463
Satan Triumphant               45464
Queerama                       45465
Length: 45466, dtype: int64

In [ ]:
# Viết hàm nhận vào index của bộ phim trả về tên của bộ phim
def get_title(index):
  Title = movies.iloc[index]['title']
  return Title

In [ ]:
get_title(4)

'Father of the Bride Part II'

In [ ]:
# Viết hàm trả và top 10 bộ phim sẽ đem đi khuyến nghị cho người dùng U. Người dung U đã xem bộ phim có tên là "Father of the Bride Part II"
def get_recommendations(title, cosine_sim=cosine_sim):
  idx = mapping[title]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:11]
  movie_mapping = [i[0] for i in sim_scores]
  return movies['title'].iloc[movie_mapping]

In [ ]:
get_recommendations('Father of the Bride Part II')

6306           North to Alaska
13611      The Magic of Méliès
19801                  Babbitt
6571                     Kuffs
5005                   Wendigo
7097        The Out of Towners
6793       Father of the Bride
926      It's a Wonderful Life
13928               Visioneers
5571            All Night Long
Name: title, dtype: object